In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
num_epochs = 4
batch_size = 4
learning_rate = 0.001

In [4]:
# dataset has PILImages of range [0,1]
# We transform them to Tensors of normalized range [-1,1]

transform = transforms.Compose(
    [transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]
)

In [6]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

100%|██████████| 170M/170M [03:27<00:00, 820kB/s]  


In [12]:
# implement conv network
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.fc1 = nn.Linear(in_features=16*5*5, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.fc3 = nn.Linear(in_features=60, out_features=10)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x

In [13]:
model = ConvNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [14]:
n_total_steps = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 2000 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

print('Finished Training')

Epoch [1/4], Step [2000/12500], Loss: 2.3447
Epoch [1/4], Step [4000/12500], Loss: 2.2973
Epoch [1/4], Step [6000/12500], Loss: 2.3026
Epoch [1/4], Step [8000/12500], Loss: 2.3008
Epoch [1/4], Step [10000/12500], Loss: 2.2848
Epoch [1/4], Step [12000/12500], Loss: 2.3028
Epoch [2/4], Step [2000/12500], Loss: 2.1738
Epoch [2/4], Step [4000/12500], Loss: 2.3985
Epoch [2/4], Step [6000/12500], Loss: 2.0128
Epoch [2/4], Step [8000/12500], Loss: 1.6971
Epoch [2/4], Step [10000/12500], Loss: 2.6026
Epoch [2/4], Step [12000/12500], Loss: 1.9857
Epoch [3/4], Step [2000/12500], Loss: 1.8726
Epoch [3/4], Step [4000/12500], Loss: 1.5688
Epoch [3/4], Step [6000/12500], Loss: 2.1236
Epoch [3/4], Step [8000/12500], Loss: 1.7826
Epoch [3/4], Step [10000/12500], Loss: 1.2149
Epoch [3/4], Step [12000/12500], Loss: 1.9232
Epoch [4/4], Step [2000/12500], Loss: 1.1474
Epoch [4/4], Step [4000/12500], Loss: 1.2069
Epoch [4/4], Step [6000/12500], Loss: 1.8766
Epoch [4/4], Step [8000/12500], Loss: 1.4830
Epoc

In [15]:
# Test the model
model.eval()
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        
        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

Accuracy of the network: 44.15 %
